In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк   

In [38]:
regions = pd.read_csv('regions.csv')
starts = pd.read_csv('starts.csv')
volunteers = pd.read_csv('volunteers.csv')



volunteers = volunteers[volunteers.athlete_age < 100]
starts = starts[starts.athlete_age < 80]
starts = starts[starts.finish_time < 5000]


cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg(
    cnt_runners = ('athlete_id', 'count')
)


young_volunteers = volunteers[volunteers['athlete_age'] <= 14]

starts = starts.merge(young_volunteers[['event_name', 'event_date', 'athlete_id']], 
                      on=['event_name', 'event_date'], 
                        how='left',
                      suffixes=('', '_vol'))

starts['has_young_volunteers'] = starts['athlete_id_vol'].notna()
starts['has_young_volunteers'] =starts['has_young_volunteers'].astype(int)
starts = starts.drop(columns=['athlete_id_vol'], axis=1)





ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique()
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique()
common_athletes = set(ages_14) & set(ages_15)
starts = starts[~starts['athlete_id'].isin(common_athletes)]



starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0)
starts = starts[starts['athlete_age'] > 14]

volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)  
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')  

starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было


unique_dates = (
    pd.concat([
        starts[['athlete_id', 'event_date']],
        volunteers[['athlete_id', 'event_date']]
    ])
    .drop_duplicates()
)
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count')
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int)
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left')


# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]  
starts = starts[starts.event_date <= '2023-10-15']


newbie = starts[starts['is_newbie'] == 1]
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id')
didnt_come = first_runs[first_runs.is_repeated == 0]

come = first_runs[first_runs.is_repeated == 1]
come_with_didnt_come = pd.concat([come, didnt_come])
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1)
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left")
print(len(come_with_didnt_come))

16708


# 0. Корреляции с is_repeated

In [39]:
numeric_cor = come_with_didnt_come.select_dtypes(include=['number'])
print(numeric_cor.corr(method='spearman')['is_repeated'].sort_values(ascending=False))

is_repeated                          1.000000
rain.1h                              0.186055
athlete_age                          0.104506
is_male                              0.035946
wind_speed                           0.031991
wind_deg                             0.030570
Составление отчёта                   0.012926
Пейсер                               0.012401
Координатор парковки                 0.008598
Сурдопереводчик                      0.005589
Организация финиша                   0.003767
Лидер для слабовидящих               0.000608
Раздача карточек позиций            -0.012038
pressure                            -0.017352
temp                                -0.018790
clouds                              -0.021863
feels_like                          -0.023001
Подготовка мероприятия              -0.024240
humidity                            -0.030813
Проведение разминки                 -0.032564
Инструктаж новых участников         -0.035377
Ведущий велосипед                 

In [40]:
display(come_with_didnt_come.head(2))

,record_id,athlete_id,event_name,event_date,city,region,finish_time,athlete_gender,athlete_age,is_newbie,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface,has_young_volunteers,is_male,Ведущий велосипед,Завершение мероприятия,Замыкающий,Инструктаж новых участников,Координатор парковки,Координация волонтёров,Лидер для слабовидящих,Маршал,Обработка результатов,Организатор,Организация финиша,Пейсер,Подготовка мероприятия,Помощь в раздаче карточек позиций,Проведение разминки,Проверка карточек позиций,Проверка трассы,Раздача карточек позиций,Разное,Связи с общественностью,Секундомер,Сканирование штрих-кодов,Сортировка карточек,Составление отчёта,Сурдопереводчик,Фотограф,Хранение и доставка оборудования,is_repeated,cnt_volunteers,cnt_runners
0,4827,790126879,Мещерский,2023-01-01,Одинцово,Московская область,2029,женский,38,1,0.85,-3.85,1007,100,0.85,100,5.0,200,Drizzle,light intensity drizzle,NaN,NaN,"гранитная крошка, грунт",1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,3.0,1,14.0,42
1,4831,790087643,Губайловский Экопарк,2023-01-01,Красногорск,Московская область,1509,мужской,15,1,0.65,-4.63,1006,100,0.65,100,6.0,200,Clouds,overcast clouds,NaN,NaN,асфальт,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1,7.0,31


# 1.0. Общий катбуст для кол-ва волонтеров

In [48]:
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split



X = come_with_didnt_come.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie',  'is_newbie',
   # 'athlete_gender', 'weather.main','weather.description','track_description_surface',
 #   'dew_point',
    'is_male',
    'cnt_volunteers',
    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = come_with_didnt_come['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0  ,cat_features=categorical_cols)  
print('катбуст начальный со всеми переменными')
model.fit(X_train, y_train)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)


катбуст начальный со всеми переменными
[[ 695  835]
 [ 425 1387]]
Accuracy: 0.62


,Feature,Importance
2,athlete_age,11.858545
0,finish_time,11.600817
44,cnt_runners,8.455844
9,wind_speed,5.985962
7,dew_point,5.310790
5,pressure,5.288004
10,wind_deg,4.830016
3,temp,4.695280
6,humidity,4.206938
4,feels_like,4.013076


# 1.1. Общий катбуст для has_volunteer и корреляции с is_repeated и has_volunteer

In [45]:
volunteer_columns = come_with_didnt_come.columns[25:52]  

# Создаем новый DataFrame для хранения результатов  
has_volunt = come_with_didnt_come.copy()  

# Перебираем роли волонтеров  
for column in volunteer_columns:  
    role_name = str(column) # Извлекаем имя роли  
    new_column_name = f'has_{role_name}'  # Создаем новое название колонки  
    has_volunt[new_column_name] = (come_with_didnt_come[column] > 0).astype(int)  # Устанавливаем 1, если есть волонтеры этой роли, иначе 0  

# Удаляем старые колонки с ролями, если требуется  
has_volunt = has_volunt.drop(columns=volunteer_columns)  
has_volunt_numeric = has_volunt.select_dtypes(include=['number'])
display(has_volunt_numeric.corr(method='spearman')['is_repeated'].sort_values(ascending=False))

is_repeated                              1.000000
rain.1h                                  0.186055
athlete_age                              0.104506
is_male                                  0.035946
wind_speed                               0.031991
wind_deg                                 0.030570
has_Пейсер                               0.013480
has_Составление отчёта                   0.012949
has_Координатор парковки                 0.008598
has_Проведение разминки                  0.007960
has_Сурдопереводчик                      0.005589
has_Организация финиша                   0.004868
has_Лидер для слабовидящих               0.000556
has_Замыкающий                          -0.011619
has_Организатор                         -0.014955
pressure                                -0.017352
has_Раздача карточек позиций            -0.018588
temp                                    -0.018790
has_Сканирование штрих-кодов            -0.020241
has_Секундомер                          -0.021316


In [47]:
X = has_volunt.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie',  'is_newbie',
   # 'athlete_gender', 'weather.main','weather.description','track_description_surface',
 #   'dew_point',
    'is_male',
    'cnt_volunteers',
    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = has_volunt['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0  ,cat_features=categorical_cols)  
print('катбуст начальный со всеми переменными для has_volunteer')
model.fit(X_train, y_train)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)

катбуст начальный со всеми переменными для has_volunteer
[[ 687  843]
 [ 432 1380]]
Accuracy: 0.62


,Feature,Importance
17,cnt_runners,15.595446
2,athlete_age,11.167075
0,finish_time,11.139378
9,wind_speed,6.400398
3,temp,6.024164
7,dew_point,5.906330
10,wind_deg,5.488667
5,pressure,5.465470
6,humidity,4.962990
4,feels_like,4.450594


///////////////////

# 1 ЧАСТЬ.   
 ВСЕ КАТБУСТЫ И МОДЕЛИ ПО ГРУППАМ (КОЛ-ВО УЧАСТНИКОВ), КАК ПРЕДИКТОРЫ КОЛ-ВО ВОЛОНТЕРОВ (has_volunteer СНИЗУ) 

In [ ]:
# ТУТ Я СЧИТАЮ КАТБУСТЫ ДЛЯ ГРУПП ЧТОБЫ ПОТОМ В МОДЕЛЬ ПИХАТЬ
lower_bound, upper_bound = 2, 46



X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= lower_bound) & (come_with_didnt_come.cnt_runners <= upper_bound)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= lower_bound) & (come_with_didnt_come.cnt_runners <= upper_bound)]
X = X.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie',
                    'cnt_runners',
   # 'athlete_gender', 'weather.main','weather.description','track_description_surface',
 #   'dew_point',
    'is_male',
    'cnt_volunteers',
    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = y['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0  ,cat_features=categorical_cols)  
print(f'катбуст для группы с кол-вом участников от {lower_bound} до {upper_bound}')

model.fit(X_train, y_train)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns, 
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)

катбуст для группы с кол-вом участников от 2 до 46
[[257 391]
 [196 779]]
Accuracy: 0.64


,Feature,Importance
2,athlete_age,12.208669
0,finish_time,11.801514
9,wind_speed,7.187620
10,wind_deg,6.759396
7,dew_point,6.685249
5,pressure,6.081936
3,temp,5.279653
6,humidity,5.041331
4,feels_like,4.382499
15,track_description_surface,4.033851


In [69]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]



#need_columns = ['athlete_age', 'finish_time', 'wind_speed','wind_deg',  'pressure',  'humidity', 'feels_like',
 #               'Маршал', 'Фотограф', 'Подготовка мероприятия', 'Связи с общественностью', 'Хранение и доставка оборудования', 'Секундомер', 'Замыкающий','Проведение разминки',
  #              'Проверка трассы', 'Завершение мероприятия', 'Сортировка карточек', 'Обработка результатов', 'Пейсер', 'has_young_volunteers', 'Сканирование штрих-кодов',
   #             'Раздача карточек позиций', 'Разное', 'Проверка карточек позиций', 'Организация финиша', 'Координация волонтёров', 'Помощь в раздаче карточек позиций',
    #            'Инструктаж новых участников', 'Составление отчёта'
                
              #   'is_male',
          #     'finish_time', 'athlete_age', 'feels_like', 'wind_speed', 'humidity'
     #         ]
need_columns = X.drop(columns=['Сурдопереводчик', 'Координатор парковки', 'Составление отчёта', 'Координация волонтёров', 
'athlete_gender','weather.main', 'weather.description', 'rain.1h', 'snow.1h', 'track_description_surface', 
'is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie', 
                    'cnt_runners', 'feels_like', 'dew_point'
], axis=1).columns.tolist()


for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.7:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]
for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')
y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

Optimization terminated successfully.
         Current function value: 0.658793
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 8113
Model:                          Logit   Df Residuals:                     8078
Method:                           MLE   Df Model:                           34
Date:                Mon, 09 Dec 2024   Pseudo R-squ.:                 0.02647
Time:                        23:29:45   Log-Likelihood:                -5344.8
converged:                       True   LL-Null:                       -5490.1
Covariance Type:            nonrobust   LLR p-value:                 1.600e-42
                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------